In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dest_idx = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dest_idx]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dest_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [11]:
mean_baseline = mean_squared_error(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
median_baseline = mean_absolute_error(y_df.values, np.repeat(np.median(y_df.values.squeeze()), y_df.shape[0]))
r2_baseline = r2_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
expl_var_baseline = explained_variance_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))

print(f"L2 baseline: {mean_baseline}")
print(f"L1 baseline: {median_baseline}")
print(f"R2 baseline: {r2_baseline}")
print(f"explained variance baseline: {expl_var_baseline}")

L2 baseline: 641687.6988062444
L1 baseline: 518.3333333333334
R2 baseline: 0.0
explained variance baseline: 0.0


In [12]:
def objective(h_params, X, y, loss_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingRegressor(
        # We use lad since it most closely matches up with hyper-parameter objective
        loss=loss_default,
        # Use this for the same reason
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [13]:
space = [
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * (len(space))
n_calls = 50 * (len(space))

In [14]:
loss_default = "ls"
scoring_default = "explained_variance"
callback_file = f"{unified_dsets[dest_idx]}_opt_gbr_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.07540275638507486, 533, 4, 'log2', 4, 1]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.1245
Function value obtained: 0.0641
Current minimum: 0.0641
Iteration No: 2 started. Evaluating function at random point.
[0.034738802219071975, 720, 5, 'log2', 4, 2]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.7964
Function value obtained: 0.0162
Current minimum: 0.0162
Iteration No: 3 started. Evaluating function at random point.
[0.08386822689969428, 343, 4, 'auto', 3, 3]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.6217
Function value obtained: 0.6375
Current minimum: 0.0162
Iteration No: 4 started. Evaluating function at random point.
[0.018853292027324983, 545, 3, 'log2', 3, 3]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.5730
Function value obtained: 0.0777
Current minimum: 0.0162
Iteration No: 5 started. Evaluating function at ra

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 108 ended. Search finished for the next optimal point.
Time taken: 2.4737
Function value obtained: -0.0039
Current minimum: -0.0889
Iteration No: 109 started. Searching for the next optimal point.
[0.001, 512, 5, 'log2', 2, 1]
Iteration No: 109 ended. Search finished for the next optimal point.
Time taken: 3.1542
Function value obtained: -0.0018
Current minimum: -0.0889
Iteration No: 110 started. Searching for the next optimal point.
[0.1, 1000, 5, 'sqrt', 6, 1]
Iteration No: 110 ended. Search finished for the next optimal point.
Time taken: 3.1372
Function value obtained: 0.1154
Current minimum: -0.0889
Iteration No: 111 started. Searching for the next optimal point.
[0.001, 410, 5, 'log2', 4, 1]
Iteration No: 111 ended. Search finished for the next optimal point.
Time taken: 2.1819
Function value obtained: 0.0055
Current minimum: -0.0889
Iteration No: 112 started. Searching for the next optimal point.
[0.001, 425, 5, 'log2', 3, 1]
Iteration No: 112 ended. Search finishe

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 120 ended. Search finished for the next optimal point.
Time taken: 3.2845
Function value obtained: -0.0028
Current minimum: -0.0889
Iteration No: 121 started. Searching for the next optimal point.
[0.001, 100, 2, 'log2', 4, 3]


...

Iteration No: 292 ended. Search finished for the next optimal point.
Time taken: 8.2994
Function value obtained: -0.0056
Current minimum: -0.0889
Iteration No: 293 started. Searching for the next optimal point.
[0.001, 540, 5, 'log2', 6, 3]
Iteration No: 293 ended. Search finished for the next optimal point.
Time taken: 10.1819
Function value obtained: -0.0159
Current minimum: -0.0889
Iteration No: 294 started. Searching for the next optimal point.
[0.001, 539, 5, 'log2', 6, 3]
Iteration No: 294 ended. Search finished for the next optimal point.
Time taken: 9.4131
Function value obtained: -0.0120
Current minimum: -0.0889
Iteration No: 295 started. Searching for the next optimal point.
[0.001, 100, 5, 'log2', 2, 1]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 295 ended. Search finished for the next optimal point.
Time taken: 9.3347
Function value obtained: 0.0014
Current minimum: -0.0889
Iteration No: 296 started. Searching for the next optimal point.
[0.001, 536, 5, 'log2', 6, 3]
Iteration No: 296 ended. Search finished for the next optimal point.
Time taken: 10.0241
Function value obtained: -0.0114
Current minimum: -0.0889
Iteration No: 297 started. Searching for the next optimal point.
[0.001, 535, 5, 'log2', 6, 3]
Iteration No: 297 ended. Search finished for the next optimal point.
Time taken: 9.5064
Function value obtained: -0.0103
Current minimum: -0.0889
Iteration No: 298 started. Searching for the next optimal point.
[0.001, 534, 5, 'log2', 6, 3]
Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 9.5441
Function value obtained: -0.0150
Current minimum: -0.0889
Iteration No: 299 started. Searching for the next optimal point.
[0.001, 408, 5, 'log2', 6, 3]
Iteration No: 299 ended. Search fini

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 11.1480
Function value obtained: -0.0150
Current minimum: -0.0889


In [15]:
loss_default = "lad"
scoring_default = "neg_mean_absolute_error"
callback_file = f"{unified_dsets[dest_idx]}_opt_gbr_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.057137099391130174, 422, 3, 'log2', 5, 3]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5476
Function value obtained: 512.7024
Current minimum: 512.7024
Iteration No: 2 started. Evaluating function at random point.
[0.0836091289729789, 988, 3, 'log2', 3, 1]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.1223
Function value obtained: 621.6607
Current minimum: 512.7024
Iteration No: 3 started. Evaluating function at random point.
[0.047918790064383154, 274, 4, 'auto', 2, 2]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.3314
Function value obtained: 596.4345
Current minimum: 512.7024
Iteration No: 4 started. Evaluating function at random point.
[0.08187791539213018, 862, 4, 'auto', 3, 1]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 9.8500
Function value obtained: 617.1078
Current minimum: 512.7024
Iteration No: 5 started. Evaluating

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 67 ended. Search finished for the next optimal point.
Time taken: 1.4139
Function value obtained: 521.4705
Current minimum: 506.8539
Iteration No: 68 started. Searching for the next optimal point.
[0.001, 100, 2, 'log2', 3, 2]
Iteration No: 68 ended. Search finished for the next optimal point.
Time taken: 1.3370
Function value obtained: 521.8552
Current minimum: 506.8539
Iteration No: 69 started. Searching for the next optimal point.
[0.001, 100, 4, 'log2', 6, 2]
Iteration No: 69 ended. Search finished for the next optimal point.
Time taken: 1.3733
Function value obtained: 521.3493
Current minimum: 506.8539
Iteration No: 70 started. Searching for the next optimal point.
[0.001, 100, 3, 'sqrt', 4, 3]
Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 2.1324
Function value obtained: 521.3015
Current minimum: 506.8539
Iteration No: 71 started. Searching for the next optimal point.
[0.001, 100, 2, 'sqrt', 4, 2]
Iteration No: 71 ended. Search finis

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 1.5784
Function value obtained: 520.1053
Current minimum: 506.8539
Iteration No: 79 started. Searching for the next optimal point.
[0.001, 100, 4, 'sqrt', 5, 3]
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 2.3208
Function value obtained: 521.1598
Current minimum: 506.8539
Iteration No: 80 started. Searching for the next optimal point.
[0.001, 472, 2, 'log2', 2, 3]
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 2.4757
Function value obtained: 518.3970
Current minimum: 506.8539
Iteration No: 81 started. Searching for the next optimal point.
[0.001, 470, 2, 'log2', 2, 2]
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.3786
Function value obtained: 517.3875
Current minimum: 506.8539
Iteration No: 82 started. Searching for the next optimal point.
[0.001, 497, 2, 'sqrt', 2, 3]
Iteration No: 82 ended. Search finis

...

Iteration No: 111 ended. Search finished for the next optimal point.
Time taken: 1.6927
Function value obtained: 521.1576
Current minimum: 506.8539
Iteration No: 112 started. Searching for the next optimal point.
[0.001, 118, 4, 'log2', 6, 2]
Iteration No: 112 ended. Search finished for the next optimal point.
Time taken: 1.7739
Function value obtained: 520.2305
Current minimum: 506.8539
Iteration No: 113 started. Searching for the next optimal point.
[0.001, 122, 4, 'log2', 6, 2]
Iteration No: 113 ended. Search finished for the next optimal point.
Time taken: 1.7632
Function value obtained: 520.9371
Current minimum: 506.8539
Iteration No: 114 started. Searching for the next optimal point.
[0.001, 1000, 2, 'sqrt', 2, 3]
Iteration No: 114 ended. Search finished for the next optimal point.
Time taken: 3.7949
Function value obtained: 515.5945
Current minimum: 506.8539
Iteration No: 115 started. Searching for the next optimal point.
[0.001, 100, 4, 'log2', 6, 2]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 115 ended. Search finished for the next optimal point.
Time taken: 3.3251
Function value obtained: 520.4081
Current minimum: 506.8539
Iteration No: 116 started. Searching for the next optimal point.
[0.004590402247603308, 100, 4, 'log2', 6, 2]
Iteration No: 116 ended. Search finished for the next optimal point.
Time taken: 1.8102
Function value obtained: 513.7191
Current minimum: 506.8539
Iteration No: 117 started. Searching for the next optimal point.
[0.002476839992425441, 100, 4, 'log2', 6, 2]
Iteration No: 117 ended. Search finished for the next optimal point.
Time taken: 1.9013
Function value obtained: 519.4183
Current minimum: 506.8539
Iteration No: 118 started. Searching for the next optimal point.
[0.005825578666500115, 100, 4, 'log2', 6, 2]
Iteration No: 118 ended. Search finished for the next optimal point.
Time taken: 1.9618
Function value obtained: 515.3755
Current minimum: 506.8539
Iteration No: 119 started. Searching for the next optimal point.
[0.0059231151

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 293 ended. Search finished for the next optimal point.
Time taken: 9.8583
Function value obtained: 510.2455
Current minimum: 502.9113
Iteration No: 294 started. Searching for the next optimal point.
[0.016090740193243767, 100, 4, 'log2', 6, 2]
Iteration No: 294 ended. Search finished for the next optimal point.
Time taken: 8.9501
Function value obtained: 517.7484
Current minimum: 502.9113
Iteration No: 295 started. Searching for the next optimal point.
[0.016021468575271067, 100, 4, 'log2', 6, 2]
Iteration No: 295 ended. Search finished for the next optimal point.
Time taken: 8.6571
Function value obtained: 514.9220
Current minimum: 502.9113
Iteration No: 296 started. Searching for the next optimal point.
[0.01610338497478621, 100, 4, 'log2', 6, 2]
Iteration No: 296 ended. Search finished for the next optimal point.
Time taken: 8.0497
Function value obtained: 514.4102
Current minimum: 502.9113
Iteration No: 297 started. Searching for the next optimal point.
[0.01621210910